
Alembic の初期設定を行い、最初のテーブルを作成するまでの手順

1. 各種ファイルを作成
    ```console
    $ alembic init migrations
    ```
2. `models.py` を作成する
3. `migrations/env.py` を設定する
    - `target_metadata` の値に `models.py` で作成したベースモデルを指定する
    - DB 接続情報を環境変数から取得できるようにする
4. マイグレーションファイルを生成する
    ```console
    $ alembic revision --autogenerate -m "Create tables" --rev-id $(date -u +"%Y%m%d_%H%M%S")
    ```
5. マイグレーションファイルを適用する
    ```console
    $ alembic upgrade head
    ```

In [9]:
from sqlalchemy.orm import Session
from sqlalchemy import select, text
from models import User, Article, Comment
from database import fetch_all, engine

In [10]:
# 接続確認
fetch_all(select(text("1")))

[1]

## データ追加

In [28]:
from datetime import datetime, timedelta

with Session(engine) as session:
    # User
    users = [
        User(name="Alice"),
        User(name="Bob"),
        User(name="Carol"),
    ]
    session.add_all(users)
    session.flush()  # ID を確定させる
    alice, bob, carol = users

    # Article
    articles = [
        Article(
            author_id=alice.id,
            title="Hello, SQLAlchemy 2.x",
            body="This is a dummy article about SQLAlchemy 2.x typed ORM.",
            published_at=datetime.now() - timedelta(days=2),
        ),
        Article(
            author_id=alice.id,
            title="Alembic autogenerate tips",
            body="A few practical tips for Alembic migrations.",
            published_at=datetime.now() - timedelta(days=2),
        ),
        Article(
            author_id=bob.id,
            title="JSONB patterns in Postgres",
            body="Dummy content: indexing JSONB, querying, and constraints.",
            published_at=None,
        ),
    ]
    session.add_all(articles)
    session.flush()
    a1, a2, a3 = articles

    # Comment
    comments = [
        Comment(
            article_id=a1.id,
            author_id=bob.id,
            body="yayayay",
        ),
        Comment(
            article_id=a1.id,
            author_id=carol.id,
            body="wowowow",
        ),
    ]
    session.add_all(comments)

    session.commit()